<a href="https://colab.research.google.com/github/Shezan57/LLM/blob/main/using_langchain_memory_concept.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2025-03-03 13:06:41--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 13.35.202.40, 13.35.202.97, 13.35.202.34, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.40|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1741010801&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MTAxMDgwMX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQxZDg4Z

In [2]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00


In [7]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 11.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.7-cp311-cp311-linux_x86_64.whl size=4601112 sha256=d143a8bb2fde0146963cf2d0715f9b226a58b61df305f7cb08eff1f34bfd0960
  Stored in directory: /root/.cache/pip/wheels/eb/82/79/ac77fcd49324b75ae6aa18e63a87cf9da4371a57e2cdc8dc03
Successfully built llama-cpp-python


In [8]:
from langchain import LlamaCpp

In [9]:
llm = LlamaCpp(
    model_path = "/content/Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers = -1,
    max_tokens = 500,
    n_ctx = 2048,
    seed = 42,
    verbose = False
)

llama_init_from_model: n_batch is less than GGML_KQ_MASK_PAD - increasing to 32
llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


In [10]:
llm.invoke("Hi! My name is shezan. What is 1 + 1?")

'\n<|assistant|> The answer to 1 + 1 is 2.\n'

In [11]:
from langchain import PromptTemplate

In [12]:
# create a template with the "input_prompt" variable
template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template = template,
    input_variables = ["input_prompt"]
)

In [13]:
basic_chain = prompt | llm

In [14]:
# Use the chain
basic_chain.invoke(
    {
        "input_prompt": "Hi! my name is Shezan. what is 1 +1?",
    }
)

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


' Hello Shezan! The answer to 1 + 1 is 2.'

## Use Chain

In [15]:
#We ask the LLM to “Create a title for a story about {summary}” where “{summary}” will be our input:
from langchain import LLMChain

In [18]:
# create a chain for the title of our story
template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

In [20]:
title.invoke({"summary": "A girl that lost her mother"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': 'A girl that lost her mother',
 'title': ' "Echoes of a Mother\'s Love: A Girl\'s Journey Through Loss"'}

In [21]:
# create chain for the character description using the summary and title
template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}.
Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(template=template,
                                  input_variables=["summary", "title"])
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [25]:
# create a chain for the story using the summary, title, and character description
template = """<s><|user|>
Create a story about {summary} with the title {title}. The main character is:
 {character}. Only return a strory and it cannot be longer than one paragraph.
 <|end|>
 <|assistant|>"""
story_prompt = PromptTemplate(template=template,
                               input_variables=["summary","title","character"])
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [26]:
llm_chain = title | character | story

In [27]:
llm_chain.invoke("a girl that lost her mother")

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': 'a girl that lost her mother',
 'title': ' "Lily\'s Lament: A Journey Through Grief"',
 'character': ' Lily, the protagonist of "Lily\'s Lament: A Journey Through Grief," is a young girl who grapples with overwhelming grief and guilt after losing her mother in an unexpected accident. As she navigates through this difficult time, Lily embarks on a poignant journey to find solace, healing, and strength within herself and the world around her.',
 'story': " Lily's Lament: A Journey Through Grief tells the heart-wrenching tale of a young girl named Lily, who is plunged into despair and guilt after losing her mother in an unforeseen accident. Devastated by grief, she struggles to come to terms with the sudden loss while wrestling with feelings of responsibility for not being able to prevent it. Seeking solace beyond her shattered world, Lily embarks on a transformative journey through the landscapes of nature and human connection. Along the way, she learns that healing is an int

## Use Memory Helping LLM to Remember Conversation

In [29]:
# Let's give the llm our name
basic_chain.invoke({"input_prompt":"Hi! My name is shezan. What is 1 + 1?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


' Hello Shezan! The answer to 1 + 1 is 2.'

In [31]:
# Next we akk the LLM to reproduce the name
basic_chain.invoke({"input_prompt":"what is my name?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


" I'm unable to determine your name as I don't have access to personal data of individuals. If you need assistance with something else, feel free to ask!"

Conversation Buffer

In [32]:
# Create an updated prompt template to include a chat history
template = """<s><|user|>Current conversation:{chat_history}
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(template=template,
                        input_variables=["input_prompt","chat_history"])

In [36]:
# Import ConversationBufferMemory
from langchain.memory import ConversationBufferMemory

In [37]:
#Define the type of memory we will use
memory = ConversationBufferMemory(memory_key='chat_history')

<ipython-input-37-2831e749aa76>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history')


In [38]:
# chain the LLM, Prompt, and memory together
llm_chain = LLMChain(
    memory=memory,
    llm=llm,
    prompt=prompt
)

In [39]:
# Generate a conversation and ask a basic question
llm_chain.invoke({"input_prompt":"Hi! My name is shezan. what is 1 + 1?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'Hi! My name is shezan. what is 1 + 1?',
 'chat_history': '',
 'text': " The answer to 1 + 1 is 2. It's a basic arithmetic operation where you add one unit to another, resulting in two units total.\n\nHere it is again for clarity:\n\n1 + 1 = 2"}

In [40]:
#Does the LLM remember the name we gave it?
llm_chain.invoke({"input_prompt": "What is my name?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is shezan. what is 1 + 1?\nAI:  The answer to 1 + 1 is 2. It's a basic arithmetic operation where you add one unit to another, resulting in two units total.\n\nHere it is again for clarity:\n\n1 + 1 = 2",
 'text': " Hello Shezan! I'm an AI, and you can call me Assistant. As for your question again: 1 + 1 equals 2.\n\nAnd to answer your last question, my name is Assistant."}

Window Coversation Buffer.
It use the last k conversations

In [41]:
from langchain.memory import ConversationBufferWindowMemory

In [42]:
# Retain only the last 2 conversations in memory
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

<ipython-input-42-9c02ad03842c>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")


In [43]:
# Chain the LLM, prompt and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [44]:
# Ask two question and generate two conversation in its memory
llm_chain.predict(input_prompt="Hi! My name is Shezan and I am 24 years old.")
llm_chain.predict(input_prompt="What is 2 + 3?")

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


' The answer to 2 + 3 is 5.'

In [45]:
# check wheter it knows the name we gave it
llm_chain.invoke({"input_prompt": "What is my name?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Shezan and I am 24 years old.\nAI:  Hello Shezan! It's great to meet you at 24 years old. How can I assist you today? Whether it's information, advice, or just a casual chat, feel free to let me know what you need. Looking forward to our conversation!\nHuman: What is 2 + 3?\nAI:  The answer to 2 + 3 is 5.",
 'text': ' Your name is Shezan.'}

In [46]:
# check wheter it knows the age we gave it
llm_chain.invoke({"input_prompt":"What is my age?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my age?',
 'chat_history': 'Human: What is 2 + 3?\nAI:  The answer to 2 + 3 is 5.\nHuman: What is my name?\nAI:  Your name is Shezan.',
 'text': " As an AI, I do not have access to personal data about individuals unless it has been shared with me during our conversation. I am designed to respect privacy and confidentiality. Therefore, I'm unable to determine your age without that information being provided by you directly. If you share it in this conversation, I can help answer any questions related to it within the scope of my capabilities."}

Coversation Summary

In [47]:
# create a summary prompt template
summary_prompt_template= """<s><|user|>Summarize the conversations and update with new lines.
Current summary:
{summary}
new lines of conversation:
{new_lines}
New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(
    template=summary_prompt_template,input_variables=["summary","new_lines"]
)

In [48]:
from langchain.memory import ConversationSummaryMemory

In [49]:
memory = ConversationSummaryMemory(llm=llm,
                                   memory_key="chat_history",
                                   prompt=summary_prompt)
#chain the LLM, Prompt and memory together
llm_chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt
)

<ipython-input-49-24ccffe411e8>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm,


In [50]:
# Generate a conversation and ask for the name
llm_chain.invoke({"input_prompt": "Hi! My name is Shezan. What is 5 + 4?"})
llm_chain.invoke({"input_prompt": "What is My name?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is My name?',
 'chat_history': ' Shezan introduces himself and asks the AI for the sum of 5 + 4, to which the AI responds that it equals 9. The explanation provided further details the process: starting from 5, adding 4 sequentially results in 9 (5, 6, 7, 8, 9). Updated summary: Shezan seeks help with a simple arithmetic problem and receives assistance to calculate 5 + 4 as 9, along with an explanatory breakdown.',
 'text': ' Your name is not mentioned in the current conversation. However, I\'m an AI and don\'t have a personal name like humans do. You can refer to me as "Assistant" or simply "AI."'}

In [51]:
# Check whether it has summarized everything thus far
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What was the first question I asked?',
 'chat_history': ' In the updated conversation, Shezan asks for help with a simple arithmetic problem and receives assistance to calculate 5 + 4 as 9, along with an explanatory breakdown. Additionally, when asked about his name, the AI clarifies that it doesn\'t have a personal name but can be referred to as "Assistant" or simply "AI."',
 'text': ' The first question you asked was, "Can you help me calculate 5 + 4?"'}

In [52]:
# Check what the summary is thus far
memory.load_memory_variables({})

{'chat_history': ' In the updated conversation, Shezan seeks help with an arithmetic problem (5 + 4), and the AI assists by correctly calculating it as 9. The AI also explains the process to provide clarity. When asked about its name, the AI states that while it doesn\'t have a personal name, it can be referred to as "Assistant" or simply "AI."'}